# Регрессия KNN и SVM

In [15]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np
from pprint import pprint, pformat
import copy
import zipfile
from pathlib import Path
import time


import missingno as msno
import joblib

from my_lib import *
from my_config import *

In [16]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, PolynomialFeatures
from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error, f1_score

from sklearn.linear_model import LinearRegression # для построения моделей линейной регрессии
from sklearn.preprocessing import PolynomialFeatures # для преобразования исходных признаков в полиномиальные, для построения моделей полиномиальной регрессии

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.svm import SVC, LinearSVC, NuSVC, SVR, NuSVR
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error

In [17]:
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 50) # Устанавливаем максимальное количество отображаемых столбцов равным 50
#pd.set_option('display.max_rows', 50) # Устанавливаем максимальное количество отображаемых строк равным 20
pd.options.display.float_format = '{:.5f}'.format # Устанавливаем формат отображения чисел с двумя знаками после запятой
pd.options.mode.use_inf_as_na = True # Настройка режима Pandas для рассмотрения бесконечностей (inf) как пропущенных значений (NA)

# Конфигурация формата отображения графиков в виде векторных изображений
%config InlineBackend.figure_format = 'svg'

# для построения графиков внутри Jupyter Notebook
%matplotlib inline

In [18]:
# Загрузить датасет подготовленный в рамках ДЗ по полиноминальной регрессии
dataset_df = joblib.load(Path(dataset_foler, dataset_filename_after_PrepareTarget))
params = joblib.load(Path(dataset_foler, params_filename_after_PrepareTarget))
print(len(dataset_df))

# сделаем 10%-ю случайную выборку, просто чтобы считалось быстрее
# надо бы что-то более хитрое, но для учебных целей это не имеет значения
dataset_df = dataset_df.sample(int(len(dataset_df)/10), random_state=42)
print(len(dataset_df))


1150653
115065


In [19]:
dataset_df_X  = dataset_df.drop([params["target_column"]], axis=1)
dataset_df_Y  = dataset_df[params["target_column"]]
display(dataset_df_X.sample(5))
display(dataset_df_Y.sample(5))

,Age,Annual Income,Number of Dependents,Education Level,Health Score,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Customer Feedback,Exercise Frequency,Policy Start Year,Policy Start Month,Policy Start Day,Policy Start Weekday,Gender_Male,Marital Status_Married,Marital Status_Single,Occupation_Self-Employed,Occupation_Unemployed,Location_Suburban,Location_Urban,Property Type_Condo,Property Type_House,Smoking Status_Yes
909876,50.00000,7073.00000,1.00000,3.00000,9.76074,2.00000,0.00000,9.00000,434.00000,8.00000,2.00000,1.00000,2021,1,9,5,1.00000,1.00000,0.00000,1.00000,0.00000,1.00000,0.00000,0.00000,1.00000,0.00000
811537,58.00000,14094.00000,4.00000,2.00000,13.29231,2.00000,0.00000,8.00000,718.00000,5.00000,1.00000,2.00000,2021,8,16,0,1.00000,0.00000,1.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1176791,26.00000,13982.00000,2.00000,0.00000,28.16551,1.00000,0.00000,15.00000,431.00000,3.00000,2.00000,1.00000,2024,2,5,0,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,1.00000,0.00000,1.00000
914875,52.00000,16054.00000,4.00000,2.00000,51.82289,2.00000,0.00000,10.00000,757.00000,4.00000,0.00000,2.00000,2021,1,13,2,1.00000,0.00000,0.00000,1.00000,0.00000,0.00000,1.00000,1.00000,0.00000,1.00000
547452,23.00000,16054.00000,4.00000,0.00000,16.88009,2.00000,1.00000,18.00000,431.00000,2.00000,2.00000,3.00000,2022,4,9,5,1.00000,1.00000,0.00000,1.00000,0.00000,0.00000,1.00000,0.00000,1.00000,0.00000


884978     115.00000
224909     966.00000
916788    1842.00000
1094768     29.00000
426657     692.00000
Name: Premium Amount, dtype: float64

In [20]:
X_train, X_test, y_train, y_test = train_test_split(dataset_df_X, dataset_df_Y, train_size=0.5, random_state=42)

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(np.array(y_train).reshape(-1, 1)).ravel()

X_test_scaled = scaler_X.transform(X_test)


In [21]:
data_for_gridsearch = {
   'X_train_scaled': X_train_scaled,
   'y_train_scaled': y_train_scaled
}
joblib.dump(data_for_gridsearch, Path(result_foler, data_for_gridsearch_filename), compress=3)

['/home/igel/Projects/ml/ml-inno-hw/3. Machine Learning/3.06. Регрессия SVM/result/data_for_gridsearch.joblib']

In [22]:
from multiprocessing import Process

def process_dataset(name, regressor_type_name, param_grid, data_for_train_filename, n_iter):
    
    # фиксируем время старта работы кода
    start = time.monotonic()

    print(f'start {name}')
    data_for_gridsearch = joblib.load(Path(result_foler, data_for_train_filename))
    
    regressor = globals()[regressor_type_name]()
    
    grid_search_srv = RandomizedSearchCV(regressor, param_grid, scoring='r2', cv=5, verbose=10, n_iter=n_iter, n_jobs=6)        
    grid_search_srv.fit(data_for_gridsearch["X_train_scaled"], data_for_gridsearch["y_train_scaled"])

    #фиксируем время окончания работы кода
    finish = time.monotonic()
    
    grif_research_result = {
        'best_params': grid_search_srv.best_params_,
        'score': grid_search_srv.best_score_,
        'runtime': finish - start
    }
    
    joblib.dump(grif_research_result, Path(result_foler, gridsearch_research_filename_mask % name), compress=3)
    print(f'finish {name}')
    

In [23]:
param_grid_rbf1 = {'gamma': ['scale', 'auto', 0.1, 1, 2],
                   'epsilon': [0.01, 0.1, 0.5, 1],
                   'C': [0.01, 0.1, 0.5],
                   'kernel': ['rbf']
                  }
param_grid_rbf2 = {'gamma': ['scale', 'auto', 0.1, 1, 2],
                   'epsilon': [0.01, 0.1, 0.5, 1],
                   'C': [1, 5, 10],
                   'kernel': ['rbf']
                  }

param_grid_KNNR = {
    'n_neighbors': [10, 15, 18],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

process_dataset("rbf1", "SVR", param_grid_rbf1, data_for_gridsearch_filename, 30)
process_dataset("rbf2", "SVR", param_grid_rbf2, data_for_gridsearch_filename, 30)
process_dataset("knn_r", "KNeighborsRegressor", param_grid_KNNR, data_for_gridsearch_filename, 12)

start rbf1
Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV 1/5; 1/30] START C=0.1, epsilon=0.01, gamma=auto, kernel=rbf................
[CV 2/5; 1/30] START C=0.1, epsilon=0.01, gamma=auto, kernel=rbf................
[CV 3/5; 1/30] START C=0.1, epsilon=0.01, gamma=auto, kernel=rbf................
[CV 4/5; 1/30] START C=0.1, epsilon=0.01, gamma=auto, kernel=rbf................
[CV 5/5; 1/30] START C=0.1, epsilon=0.01, gamma=auto, kernel=rbf................
[CV 1/5; 2/30] START C=0.01, epsilon=1, gamma=auto, kernel=rbf..................
[CV 1/5; 2/30] END C=0.01, epsilon=1, gamma=auto, kernel=rbf;, score=-0.000 total time=  24.6s
[CV 2/5; 2/30] START C=0.01, epsilon=1, gamma=auto, kernel=rbf..................
[CV 2/5; 2/30] END C=0.01, epsilon=1, gamma=auto, kernel=rbf;, score=-0.001 total time=  23.1s
[CV 3/5; 2/30] START C=0.01, epsilon=1, gamma=auto, kernel=rbf..................
[CV 3/5; 2/30] END C=0.01, epsilon=1, gamma=auto, kernel=rbf;, score=-0.001 total time=  

In [24]:
def show_grid_search_results(grid_research_results, regressor_name, return_the_best_params=True):
    gsr = sorted(grid_research_results, key=lambda r: r['score'], reverse=True )
    print(f'Результаты поиска оптимальных гиперпараметров для {regressor_name}(). Верхняя строка - лучшая.')
    for gr in gsr:  
        print(f"score={round(gr['score'] ,3)}, runtime={round(gr['runtime'],2):7}, best_params={gr['best_params']}")
    if return_the_best_params:
        return gsr[0]['best_params'] 

names_svr = ['rbf1', 'rbf2']
grid_research_results = []
for name in names_svr:
    grid_research_results.append(joblib.load(Path(result_foler, gridsearch_research_filename_mask % name)))
best_params_svr = show_grid_search_results(grid_research_results, 'SVR')

names_knn = ['knn_r']
grid_research_results = []
for name in names_knn:
    grid_research_results.append(joblib.load(Path(result_foler, gridsearch_research_filename_mask % name)))
grid_research_results = sorted(grid_research_results, key=lambda r: r['score'], reverse=True )
print()
best_params_knn = show_grid_search_results(grid_research_results, 'KNeighborsRegressor')

Результаты поиска оптимальных гиперпараметров для SVR(). Верхняя строка - лучшая.
score=-0.0, runtime=12640.15, best_params={'kernel': 'rbf', 'gamma': 2, 'epsilon': 0.01, 'C': 10}
score=-0.0, runtime=2339.87, best_params={'kernel': 'rbf', 'gamma': 'scale', 'epsilon': 1, 'C': 0.01}

Результаты поиска оптимальных гиперпараметров для KNeighborsRegressor(). Верхняя строка - лучшая.
score=-0.049, runtime=  20.43, best_params={'weights': 'uniform', 'p': 1, 'n_neighbors': 18}


In [ ]:
# Обучаем SVM для регрессии
svm_regressor = SVR(**best_params_svr)
svm_regressor.fit(X_train_scaled, y_train_scaled)
joblib.dump(svm_regressor, Path(result_foler, model_template_filename % "svr"), compress=3)

In [31]:
y_train_pred_scaled = svm_regressor.predict(X_train_scaled)
y_train_pred = scaler_y.inverse_transform(np.array(y_train_pred_scaled).reshape(-1, 1)).ravel()

y_test_pred_scaled = svm_regressor.predict(X_test_scaled)
y_test_pred = scaler_y.inverse_transform(np.array(y_test_pred_scaled).reshape(-1, 1)).ravel()

In [32]:
check_model2(y_train, y_train_pred, y_test, y_test_pred)

Train data:
  MSE:    49.9206
  RMSE:   7.0654
  MAE:    7.0559
  r2:     0.9999
  median: 831.0
Test data:
  MSE:    503902.2608
  RMSE:   709.8607
  MAE:    569.1775
  r2:     -0.0001
  median: 837.0


Модель переобучилась - на тренировочной выборке r2 близка к 1, а на тестовой - около 0.

In [33]:
# Обучаем KNeighborsRegressor для регрессии
knn_regressor = KNeighborsRegressor(**best_params_knn)
knn_regressor.fit(X_train_scaled, y_train_scaled)
joblib.dump(knn_regressor, Path(result_foler, model_template_filename % "knnr"), compress=3)

['/home/igel/Projects/ml/ml-inno-hw/3. Machine Learning/3.06. Регрессия SVM/result/model_knnr.pickle']

In [34]:

y_train_pred_scaled = knn_regressor.predict(X_train_scaled)
y_train_pred = scaler_y.inverse_transform(np.array(y_train_pred_scaled).reshape(-1, 1)).ravel()

y_test_pred_scaled = knn_regressor.predict(X_test_scaled)
y_test_pred = scaler_y.inverse_transform(np.array(y_test_pred_scaled).reshape(-1, 1)).ravel()

In [35]:
check_model2(y_train, y_train_pred, y_test, y_test_pred)

Train data:
  MSE:    470586.4098
  RMSE:   685.993
  MAE:    548.0749
  r2:     0.061
  median: 831.0
Test data:
  MSE:    531020.9404
  RMSE:   728.7118
  MAE:    581.3144
  r2:     -0.0539
  median: 837.0


Качество модели плохое. R2 близко к нулю, для предсказания целевой переменной использовать нельзя.